In [35]:
#Импортируем либы: 
import pandas as pd
import numpy as np

import calendar

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(
    font_scale =2,
    style      ="whitegrid",
    rc         ={'figure.figsize':(20,7)}
)

In [12]:
#Импортируем датасеты:
customers = pd.read_csv("/mnt/HC_Volume_18315164/home-jupyter/jupyter-k-stojanov-21/first_project_datasets/olist_customers_dataset.csv")

orders = pd.read_csv("/mnt/HC_Volume_18315164/home-jupyter/jupyter-k-stojanov-21/first_project_datasets/olist_orders_dataset.csv")

items = pd.read_csv("/mnt/HC_Volume_18315164/home-jupyter/jupyter-k-stojanov-21/first_project_datasets/olist_order_items_dataset.csv")

# Предварительное исследование:

Число клиентов

In [4]:
customers.customer_unique_id.nunique()

96096

статусы заказа:

In [6]:
orders.order_status.unique()

array(['delivered', 'invoiced', 'shipped', 'processing', 'unavailable',
       'canceled', 'created', 'approved'], dtype=object)

Статусы которые можно отметить "закрытыми" - при которых покупка совершилась:
  
`delivered, shipped, approved` -   
доставлен, отправлен, подтвержден

**статусы неудачи:**   
`unavailable, canceled`  
недоступен, отменен

**промежуточные статусы:**   
`created, invoiced, processing`  
создан, выставлнен счёт, обрабатывается

Число заказов:

In [8]:
orders.order_id.count()

99441

Период наблюдений:

In [13]:
print('Orders from {} to {}'.format(orders['order_purchase_timestamp'].min(),
                                    orders['order_purchase_timestamp'].max()))

Orders from 2016-09-04 21:15:19 to 2018-10-17 17:30:18


**География**

судя по всему площадка (так как есть более одного магазина в ней) находится в Бразилии в штате Сан Паулу, собственно:  
sp - Сан Паулу   
rj Рио-де-Жанейро    
mg Минас-Жерайс  
и так далее  

**Число клиентов по штатам:**

In [21]:
customers.groupby("customer_state").customer_unique_id.count() \
         .reset_index().sort_values("customer_unique_id", ascending = False)

,customer_state,customer_unique_id
25,SP,41746
18,RJ,12852
10,MG,11635
22,RS,5466
17,PR,5045
23,SC,3637
4,BA,3380
6,DF,2140
7,ES,2033
8,GO,2020


# 1. Сколько у нас пользователей, которые совершили покупку только один раз? (7 баллов) 

In [18]:
#выбираем походящие статусы решил добавить сюда 
# processing - как я понял это именно обработка платежа после оплаты
buy = ['delivered', 'shipped', 'processing',
       'approved']

Вбрал те, что близки к завершению или оплачены (при оплате можно отменить, но число отмен после оплаты - отдельный вопрос)

In [19]:
#Женим таблицы так как нам нужны аггрегейты по пользователям. 
#Райт джойн, чтобы не потреять именно заказы
cust_order = customers.merge(orders, how = 'right', on = 'customer_id')

In [51]:
cust_order.query("order_status in @buy") \
          .groupby(['customer_unique_id']) \
          .order_id.count().reset_index().query("order_id == 1").shape[0]

93099

Собственно ответ: **91816** (в черновике есть ещё другой вариант решения)

# 2. Сколько заказов в месяц в среднем не доставляется по разным причинам (вывести детализацию по причинам)?  (5 баллов)

In [22]:
orders.order_purchase_timestamp = pd.to_datetime(orders.order_purchase_timestamp)

In [23]:
orders["order_purchase_month"] = orders.order_purchase_timestamp.dt.month
orders["order_purchase_year"] = orders.order_purchase_timestamp.dt.year

In [24]:
#выбираем походящие
not_delivered = ['invoiced', 'shipped', 'processing','unavailable',
       'canceled', 'created', 'approved']

In [26]:
orders.query("order_status == 'canceled'") \
    .groupby(["order_purchase_year", "order_purchase_month"], as_index =False) \
    .order_id.nunique().sort_values(["order_purchase_year", "order_purchase_month"])

,order_purchase_year,order_purchase_month,order_id
0,2016,9,2
1,2016,10,24
2,2017,1,3
3,2017,2,17
4,2017,3,33
5,2017,4,18
6,2017,5,29
7,2017,6,16
8,2017,7,28
9,2017,8,27


**Детализация по причинам**

In [27]:
orders.query("order_status in  @not_delivered") \
    .groupby(["order_purchase_month", "order_purchase_year", "order_status"], as_index =False) \
    .order_id.nunique().sort_values(["order_purchase_year", "order_purchase_month"])

,order_purchase_month,order_purchase_year,order_status,order_id
81,9,2016,canceled,2
82,9,2016,shipped,1
90,10,2016,canceled,24
91,10,2016,invoiced,18
92,10,2016,processing,2
...,...,...,...,...
79,8,2018,shipped,47
80,8,2018,unavailable,7
88,9,2018,canceled,15
89,9,2018,shipped,1


# 3. По каждому товару определить, в какой день недели товар чаще всего покупается. (5 баллов)

In [28]:
order_item = orders.merge(items, on = 'order_id')

In [29]:
order_item["order_purchase_weekday"] = orders.order_purchase_timestamp.dt.day_name()

## Первый вариант, через моду (самый правильный):
*долго исполняется*

In [32]:
order_item.groupby("product_id").order_purchase_weekday.apply(pd.Series.mode)

product_id                         
0009406fd7479715e4bef61dd91f2462  0     Saturday
000b8f95fcb9e0096488278317764d19  0       Monday
000d9be29b5207b54e86aa1b1ac54872  0       Monday
0011c512eb256aa0dbbb544d8dffcf6e  0      Tuesday
00126f27c813603687e6ce486d909d01  0       Friday
                                         ...    
fff81cc3158d2725c0655ab9ba0f712c  0    Wednesday
fff9553ac224cec9d15d49f5a263411f  0       Monday
fffdb2d0ec8d6a61f0a0a0db3f25b441  0       Friday
                                  1     Thursday
fffe9eeff12fcbd74a2f2b007dde0c58  0       Monday
Name: order_purchase_weekday, Length: 41804, dtype: object

## Второй вариант со счётчиком: 

In [30]:
#слишком громоздкий вариант
order_item.groupby(["product_id", "order_purchase_weekday"], as_index=False) \
    .order_id.count().groupby('product_id') \
    .apply(lambda x: x.order_id[x.order_id.idxmax()]).to_frame() \
    .merge(  order_item.groupby(["product_id", "order_purchase_weekday"], as_index=False) \
            .order_id.count().groupby('product_id') \
            .apply(lambda x: x.order_purchase_weekday[x.order_id.idxmax()]) \
            .to_frame(),
    on = "product_id") \
    .sort_values("0_x", ascending = False)

,0_x,0_y
product_id,,
aca2eb7d00ea1a7b8ebd4e68314663af,81,Monday
422879e10f46682990de24d770e7f83d,79,Monday
99a4788cb24856965c36a24e339b6058,75,Tuesday
53759a2ecddad2bb87a079a1f1519f73,62,Monday
368c6c730842d78016ad823897a372db,61,Thursday
...,...,...
5f76fa92c238431e69b3f294ec7db04a,1,Friday
5f76bb88bf4410580362879247ef7248,1,Tuesday
5f754b00e7c346bd070d33ffb9f006a7,1,Monday


## Третий вариант без счётчика:

In [31]:
order_item.groupby(["product_id", "order_purchase_weekday"], as_index=False) \
    .order_id.count().groupby('product_id').apply(lambda x: x.order_purchase_weekday[x.order_id.idxmax()])

product_id
0009406fd7479715e4bef61dd91f2462     Saturday
000b8f95fcb9e0096488278317764d19       Monday
000d9be29b5207b54e86aa1b1ac54872       Monday
0011c512eb256aa0dbbb544d8dffcf6e      Tuesday
00126f27c813603687e6ce486d909d01       Friday
                                      ...    
fff6177642830a9a94a0f2cba5e476d1       Friday
fff81cc3158d2725c0655ab9ba0f712c    Wednesday
fff9553ac224cec9d15d49f5a263411f       Monday
fffdb2d0ec8d6a61f0a0a0db3f25b441       Friday
fffe9eeff12fcbd74a2f2b007dde0c58       Monday
Length: 30619, dtype: object

# 4. Сколько у каждого из пользователей в среднем покупок в неделю (по месяцам)? Не стоит забывать, что внутри месяца может быть не целое количество недель. Например, в ноябре 2021 года 4,28 недели. И внутри метрики это нужно учесть. (8 баллов) 

*Короче, если я правильно понял нужно сгруппировать по месяцам, посчитать число заказов в месяц, затем поделить на среднее число недель в месяце (примерно 4, но на самом деле нет - смотри ниже)*  

In [33]:
30/7

4.285714285714286

In [34]:
31/7

4.428571428571429

In [37]:
cust_order.order_purchase_timestamp = pd.to_datetime(cust_order.order_purchase_timestamp)

In [38]:
#разделю таймстемпы по столбцам (для удобства)
cust_order["order_purchase_month"] = cust_order.order_purchase_timestamp.dt.month
cust_order["order_purchase_year"] = cust_order.order_purchase_timestamp.dt.year

In [39]:
# глянем, проверим
cust_order[["order_purchase_timestamp", "order_purchase_year","order_purchase_month"]].head()

,order_purchase_timestamp,order_purchase_year,order_purchase_month
0,2017-10-02 10:56:33,2017,10
1,2018-07-24 20:41:37,2018,7
2,2018-08-08 08:38:49,2018,8
3,2017-11-18 19:28:06,2017,11
4,2018-02-13 21:18:39,2018,2


#### Нужно посчитать количество недель в месяцах:
Сначала на примере одного месяца:

In [40]:
print(calendar.month(2021,11))

   November 2021
Mo Tu We Th Fr Sa Su
 1  2  3  4  5  6  7
 8  9 10 11 12 13 14
15 16 17 18 19 20 21
22 23 24 25 26 27 28
29 30



In [41]:
print(calendar.monthrange(2021, 11)[1] / 7)

4.285714285714286


## Первый вариант решения: (без предварительной группировки)

In [42]:
#посчитаем количество недель в каждом месяце число дней / на 7
for i, j in cust_order[['order_purchase_year', 'order_purchase_month']].itertuples(index=False):
    cust_order["num_of_week"] = calendar.monthrange(i, j)[1] / 7

In [43]:
#собственно поделим одно на другое:

cust_order["purchase_per_week"] = ((cust_order.groupby(['customer_unique_id', "order_purchase_month"]) \
                                   .order_id.transform("count"))
                                   / cust_order["num_of_week"])

## Второй вариант решения: (с предварительной группировкой) 
**вроде так правильнее**

In [44]:
#группируем
for_mean_purch_per_week = cust_order \
    .groupby(['customer_unique_id', 'order_purchase_year', "order_purchase_month"], as_index = False) \
    .order_id.count() 

In [45]:
# собстно решение:
for i, j in for_mean_purch_per_week[['order_purchase_year', 'order_purchase_month']].itertuples(index=False):
    for_mean_purch_per_week["week_per_month"] = calendar.monthrange(i, j)[1] / 7
    
for_mean_purch_per_week["purchase_per_week"] = (for_mean_purch_per_week.order_id / for_mean_purch_per_week.week_per_month)

In [46]:
# наведем красоту: 
for_mean_purch_per_week = for_mean_purch_per_week \
.rename(columns = {
    'order_id': 'order_count'
})

In [47]:
#Проверим
for_mean_purch_per_week.sort_values("order_count", ascending = False).head()

,customer_unique_id,order_purchase_year,order_purchase_month,order_count,week_per_month,purchase_per_week
7325,12f5d6e1cbf93dafd9dcc19095df0b3d,2017,1,6,4.428571,1.354839
50622,83e7958a94bd7f74a9414d8782f87628,2017,1,4,4.428571,0.903226
76793,c8460e4251689ba205045f3ea17884a1,2018,8,4,4.428571,0.903226
67755,b08fab27d47a1eb6deda07bfd965ad43,2017,9,4,4.428571,0.903226
62196,a239b8e2fbce33780f1f1912e2ee5275,2017,2,4,4.428571,0.903226


### Сравним Варианты решения: 

In [48]:
cust_order[["customer_unique_id", "order_purchase_timestamp", "order_purchase_year", 
            "order_purchase_month", "num_of_week",  "purchase_per_week"]] \
    .query("customer_unique_id == '12f5d6e1cbf93dafd9dcc19095df0b3d'") \
    .sort_values("purchase_per_week", ascending = False).head(10)

,customer_unique_id,order_purchase_timestamp,order_purchase_year,order_purchase_month,num_of_week,purchase_per_week
934,12f5d6e1cbf93dafd9dcc19095df0b3d,2017-01-05 15:25:10,2017,1,4.428571,1.354839
4715,12f5d6e1cbf93dafd9dcc19095df0b3d,2017-01-05 14:22:02,2017,1,4.428571,1.354839
28014,12f5d6e1cbf93dafd9dcc19095df0b3d,2017-01-05 15:25:04,2017,1,4.428571,1.354839
35818,12f5d6e1cbf93dafd9dcc19095df0b3d,2017-01-05 14:50:54,2017,1,4.428571,1.354839
81491,12f5d6e1cbf93dafd9dcc19095df0b3d,2017-01-05 14:23:54,2017,1,4.428571,1.354839
82436,12f5d6e1cbf93dafd9dcc19095df0b3d,2017-01-05 14:18:03,2017,1,4.428571,1.354839


In [49]:
for_mean_purch_per_week.query("customer_unique_id == '12f5d6e1cbf93dafd9dcc19095df0b3d'") \
    .sort_values("purchase_per_week", ascending = False).head()

,customer_unique_id,order_purchase_year,order_purchase_month,order_count,week_per_month,purchase_per_week
7325,12f5d6e1cbf93dafd9dcc19095df0b3d,2017,1,6,4.428571,1.354839


Разница в том, что в первом варианте дублируются заказы (второй удобнее)

**Продолжение в следующих ноутбуках**